In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

# Load dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize and reshape
X_train = X_train.astype("float32") / 255.0
X_test  = X_test.astype("float32") / 255.0
X_train = np.expand_dims(X_train, -1)
X_test  = np.expand_dims(X_test, -1)

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test  = tf.keras.utils.to_categorical(y_test, 10)

# CNN Model (≈185k params)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(X_train, y_train, epochs=15, batch_size=64,
                    validation_split=0.1, verbose=1)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

model.summary()

4422102/4422102 [==============================] - 1s 0us/step
Epoch 1/15
844/844 [==============================] - 16s 11ms/step - loss: 0.5102 - accuracy: 0.8139 - val_loss: 0.3745 - val_accuracy: 0.8602
Epoch 2/15
844/844 [==============================] - 8s 10ms/step - loss: 0.3218 - accuracy: 0.8820 - val_loss: 0.3163 - val_accuracy: 0.8835
Epoch 3/15
844/844 [==============================] - 8s 10ms/step - loss: 0.2741 - accuracy: 0.9006 - val_loss: 0.2723 - val_accuracy: 0.9023
Epoch 4/15
844/844 [==============================] - 9s 10ms/step - loss: 0.2431 - accuracy: 0.9097 - val_loss: 0.2677 - val_accuracy: 0.9002
Epoch 5/15
844/844 [==============================] - 9s 10ms/step - loss: 0.2181 - accuracy: 0.9185 - val_loss: 0.2441 - val_accuracy: 0.9140
Epoch 6/15
844/844 [==============================] - 8s 10ms/step - loss: 0.1941 - accuracy: 0.9289 - val_loss: 0.2761 - val_accuracy: 0.9002
Epoch 7/15
844/844 [==============================] - 8s 10ms/step - loss: 0.1

In [2]:
# Float32 model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("fashion_cnn.tflite", "wb") as f:
    f.write(tflite_model)

# Quantized int8 model
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
with open("fashion_cnn_quant.tflite", "wb") as f:
    f.write(tflite_quant_model)


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmp2791j2ug\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmp2791j2ug\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpucvz35l6\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpucvz35l6\assets


In [3]:
!xxd -i fashion_cnn_quant.tflite > fashion_cnn_quant.h